In [33]:
!feast apply

/Users/shaurya.rawat/Documents/mlplatform/feature-store/.venv/lib/python3.8/site-packages/feast/stream_feature_view.py:95: RuntimeWarning: Stream feature views are experimental features in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
/Users/shaurya.rawat/Documents/mlplatform/feature-store/.venv/lib/python3.8/site-packages/feast/infra/offline_stores/file_source.py:161: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  schema = ParquetDataset(path).schema.to_arrow_schema()
Updated feature view customer_stats
	ttl: seconds: 7776000
 -> seconds: 8640000000

Updated feature view driver_stats
	ttl: seconds: 7776000
 -> seconds: 8640000000

Updated stream feature 

In [1]:
from feast import FeatureStore
import warnings

warnings.filterwarnings("ignore")

store = FeatureStore(".")

In [2]:
list(map(lambda x: str(x), store.registry.list_entities(project="hybrid_store")))

['{\n  "spec": {\n    "name": "driver",\n    "description": "Driver ID",\n    "joinKey": "driver_id"\n  },\n  "meta": {\n    "createdTimestamp": "2023-02-01T15:52:34.143640Z",\n    "lastUpdatedTimestamp": "2023-02-01T15:52:34.143640Z"\n  }\n}',
 '{\n  "spec": {\n    "name": "customer",\n    "description": "Customer ID",\n    "joinKey": "customer_id"\n  },\n  "meta": {\n    "createdTimestamp": "2023-02-01T15:52:34.143679Z",\n    "lastUpdatedTimestamp": "2023-02-01T15:52:34.143679Z"\n  }\n}',
 '{\n  "spec": {\n    "name": "__dummy",\n    "joinKey": "__dummy_id"\n  },\n  "meta": {\n    "createdTimestamp": "2023-02-01T15:52:34.143590Z",\n    "lastUpdatedTimestamp": "2023-02-01T15:52:34.143716Z"\n  }\n}']

In [3]:
store.registry.list_stream_feature_views(project="hybrid_store")

[<StreamFeatureView(aggregations = [], mode = spark, timestamp_field = event_timestamp, udf = <function driver_hourly_stats_stream at 0x7fed48e47ee0>, udf_string = @stream_feature_view(
     entities=[driver],
     ttl=timedelta(days=100000),
     mode="spark",
     schema=[
         Field(name="conv_percentage", dtype=Float32),
         Field(name="acc_percentage", dtype=Float32),
     ],
     timestamp_field="event_timestamp",
     online=True,
     source=driver_stats_stream_source,
     tags={"stream": "True"},
 )
 def driver_hourly_stats_stream(df: DataFrame):
     # Define UDF here
     from pyspark.sql.functions import col
 
     return (
         df.withColumn("conv_percentage", col("conv_rate") * 100.0)
         .withColumn("acc_percentage", col("acc_rate") * 100.0)
         .drop("conv_rate", "acc_rate")
     )
 , name = driver_hourly_stats_stream, entities = ['driver'], ttl = 100000 days, 0:00:00, stream_source = {
   "type": "STREAM_KAFKA",
   "timestampField": "event_times

In [4]:
import pandas as pd
from datetime import datetime

entity_df = pd.DataFrame.from_dict(
    {
        "customer_id": [5001, 5002, 5003, 5004, 5005],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
            datetime(2021, 4, 12, 15, 1, 12),
            datetime.now()
        ]
    }
)

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "customer_stats:current_balance",
        "customer_stats:avg_passenger_count",
        "customer_stats:lifetime_trip_count"
    ]
).to_df()

print(training_df.head(10))

   customer_id                  event_timestamp  current_balance  \
0         5001        2021-04-12 10:59:42+00:00         0.174109   
1         5003        2021-04-12 16:40:26+00:00         0.735872   
2         5004        2021-04-12 15:01:12+00:00         0.885541   
3         5002        2021-04-12 08:12:10+00:00         0.922777   
4         5005 2023-02-02 14:01:50.296475+00:00         0.928581   

   avg_passenger_count  lifetime_trip_count  
0             0.384933                   14  
1             0.542926                  616  
2             0.774241                  129  
3             0.167704                  844  
4             0.883550                  399  


In [5]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003, 1004, 1001],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
            datetime(2021, 4, 12, 15, 1, 12),
            datetime.now()
        ]
    }
)

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats_stream:conv_percentage",
        "driver_hourly_stats_stream:acc_percentage"
    ]
).to_df()

training_df.head()

,driver_id,event_timestamp,conv_percentage,acc_percentage
0,1001,2021-04-12 10:59:42+00:00,52.114902,75.165855
1,1003,2021-04-12 16:40:26+00:00,18.885477,34.473606
2,1004,2021-04-12 15:01:12+00:00,29.649216,93.530525
3,1002,2021-04-12 08:12:10+00:00,8.901370,21.263689
4,1001,2023-02-02 14:01:50.407645+00:00,40.458847,40.757076


In [6]:
from datetime import datetime, timedelta

store.materialize_incremental(end_date=datetime.now() - timedelta(minutes=5))

Materializing 3 feature views to 2023-02-02 13:56:50+01:00 into the redis online store.

customer_stats from 2023-02-02 14:54:33+01:00 to 2023-02-02 13:56:50+01:00:


0it [00:00, ?it/s]


driver_stats from 2023-02-02 14:54:33+01:00 to 2023-02-02 14:56:50+01:00:


0it [00:00, ?it/s]


driver_hourly_stats_stream from 2023-02-02 14:54:33+01:00 to 2023-02-02 14:56:50+01:00:


0it [00:00, ?it/s]


In [7]:
# Online stores
features = store.get_online_features(
    features=[
        "driver_hourly_stats_stream:conv_percentage",
        "driver_hourly_stats_stream:acc_percentage"
    ],
    entity_rows=[
        {"driver_id": 1001},
        {"driver_id": 1002}
    ]
).to_dict(include_event_timestamps=True)

features

{'driver_id': [1001, 1002],
 'driver_id__ts': [0, 0],
 'conv_percentage': [40.45884704589844, 46.587459564208984],
 'conv_percentage__ts': [1647266400, 1647266400],
 'acc_percentage': [40.757076263427734, 31.57208824157715],
 'acc_percentage__ts': [1647266400, 1647266400]}

In [8]:
# Make sure stream  delivers fresh feature values to the online store for the entites it has been registered with
import os
from pyspark.sql import SparkSession

os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages=org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell"
spark = SparkSession.builder.master("local[*]").appName("feast-spark").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", 5)

23/02/02 14:01:54 WARN Utils: Your hostname, nb-srawat.local resolves to a loopback address: 127.0.0.1; using 192.168.1.133 instead (on interface en0)
23/02/02 14:01:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.2.1/libexec/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/shaurya.rawat/.ivy2/cache
The jars for the packages stored in: /Users/shaurya.rawat/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e2da9e3-df46-4c99-b3ea-65ec5a026806;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in local-m2-cache
	found com.github.luben#zstd-jni;1.4.4-3 in local-m2-cache
	found org.lz4#lz4-java;1.7.1 in local-m2-cache
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.spark-project.spark#unused;1.0.0 in local-m2-cache
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 4701ms :: artifacts dl 65ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.4-3 from local-m2-cache in [defaul

In [9]:
from feast.data_source import PushMode
from processor import ProcessorConfig, SparkProcessor

def preprocess_fn(rows: pd.DataFrame):
    print(f"DF cols: {rows.columns}")
    print(f"DF Size: {rows.size}")
    print(f"DF head: {rows.head(5)}")
    return rows

config = ProcessorConfig(mode="spark", spark_session=spark, processing_time="30 seconds", query_timeout=15)
stream_fv = store.get_stream_feature_view("driver_hourly_stats_stream")

processor = SparkProcessor(
    fs=store,
    sfv=stream_fv,
    config=config,
    preprocess_fn=preprocess_fn
)

query = processor.ingest_stream_into_online_store(to=PushMode.ONLINE)


23/02/02 14:02:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 0
DF head: Empty DataFrame
Columns: [driver_id, event_timestamp, created, conv_percentage, acc_percentage]
Index: []


DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 25
DF head:    driver_id     event_timestamp                          created  \
0       1001 2031-02-22 22:00:00 2023-02-02 13:02:39.297797+00:00   
1       1002 2031-02-22 21:00:00 2023-02-02 13:02:39.297797+00:00   
2       1003 2031-02-22 21:00:00 2023-02-02 13:02:39.297797+00:00   
3       1004 2031-02-22 22:00:00 2023-02-02 13:02:39.297797+00:00   
4       1005 2031-02-22 22:00:00 2023-02-02 13:02:39.297797+00:00   

   conv_percentage  acc_percentage  
0         6.938688        9.072059  
1        18.959631       54.245543  
2        96.221501       68.706465  
3         0.420751       21.136233  
4        57.497627       83.557701  
DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 25
DF head:    driver_id     event_timestamp                          created  \


In [24]:
# Online features after stream ingestion
features = store.get_online_features(
    features=[
        "driver_hourly_stats_stream:conv_percentage",
        "driver_hourly_stats_stream:acc_percentage"
    ],
    entity_rows=[
        {"driver_id": 1001},
        {"driver_id": 1003}
    ]
).to_dict(include_event_timestamps=True)

features

{'driver_id': [1001, 1003],
 'driver_id__ts': [0, 0],
 'conv_percentage': [6.938688278198242, 96.22150421142578],
 'conv_percentage__ts': [1929564000, 1929560400],
 'acc_percentage': [9.07205867767334, 68.70646667480469],
 'acc_percentage__ts': [1929564000, 1929560400]}

In [25]:
query.stop()

23/02/02 14:44:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2023276 ms exceeds timeout 120000 ms
23/02/02 14:44:35 WARN SparkContext: Killing executors is not supported by current scheduler.


In [27]:
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004, 1005],
        "event_timestamp": [
            datetime(2027, 3, 20, 10, 59, 42),
            datetime(2027, 3, 20, 10, 59, 42),
            datetime(2027, 3, 20, 10, 59, 42),
            datetime(2027, 3, 20, 10, 59, 42),
            datetime(2027, 3, 20, 10, 59, 42),
        ]
})

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats_stream:conv_percentage",
        "driver_hourly_stats_stream:acc_percentage"
    ]
).to_df()

training_df.head(10)

,driver_id,event_timestamp,conv_percentage,acc_percentage
0,1001,2027-03-20 10:59:42+00:00,40.458847,40.757076
1,1004,2027-03-20 10:59:42+00:00,97.727554,5.158236
2,1003,2027-03-20 10:59:42+00:00,86.991714,77.956184
3,1002,2027-03-20 10:59:42+00:00,46.587460,31.572088
4,1005,2027-03-20 10:59:42+00:00,39.407246,4.611785


In [28]:
query = processor.ingest_stream_into_online_store(PushMode.OFFLINE)

23/02/02 15:41:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 25
DF head:    driver_id     event_timestamp                          created  \
0       1001 2024-03-05 22:00:00 2023-02-02 14:41:45.957448+00:00   
1       1002 2024-03-05 23:00:00 2023-02-02 14:41:45.957448+00:00   
2       1003 2024-03-05 22:00:00 2023-02-02 14:41:45.957448+00:00   
3       1004 2024-03-05 22:00:00 2023-02-02 14:41:45.957448+00:00   
4       1005 2024-03-05 22:00:00 2023-02-02 14:41:45.957448+00:00   

   conv_percentage  acc_percentage  
0        16.334991       41.511256  
1        62.255239       70.562595  
2        51.607168       73.766935  
3        69.630742       60.157454  
4        68.359011       61.037648  


DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 25
DF head:    driver_id     event_timestamp                          created  \
0       1001 2024-03-06 01:00:00 2023-02-02 14:42:00.599749+00:00   
1       1002 2024-03-06 01:00:00 2023-02-02 14:42:00.599749+00:00   
2       1003 2024-03-06 02:00:00 2023-02-02 14:42:00.599749+00:00   
3       1004 2024-03-06 01:00:00 2023-02-02 14:42:00.599749+00:00   
4       1005 2024-03-06 01:00:00 2023-02-02 14:42:00.599749+00:00   

   conv_percentage  acc_percentage  
0        29.077673       10.660022  
1        11.273976       44.519717  
2        66.780871       14.266326  
3        57.489824       95.271534  
4        29.847521       11.959860  


In [29]:
# Query after ingesting into the offline store
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats_stream:conv_percentage",
        "driver_hourly_stats_stream:acc_percentage"
    ]
).to_df()

training_df.head(10)

,driver_id,event_timestamp,conv_percentage,acc_percentage
0,1004,2027-03-20 10:59:42+00:00,57.489822,95.271530
1,1005,2027-03-20 10:59:42+00:00,29.847521,11.959860
2,1002,2027-03-20 10:59:42+00:00,11.273976,44.519718
3,1001,2027-03-20 10:59:42+00:00,29.077673,10.660023
4,1003,2027-03-20 10:59:42+00:00,66.780869,14.266326


DF cols: Index(['driver_id', 'event_timestamp', 'created', 'conv_percentage',
       'acc_percentage'],
      dtype='object')
DF Size: 25
DF head:    driver_id     event_timestamp                          created  \
0       1001 2024-03-06 07:00:00 2023-02-02 14:42:30.541654+00:00   
1       1002 2024-03-06 07:00:00 2023-02-02 14:42:30.541654+00:00   
2       1003 2024-03-06 07:00:00 2023-02-02 14:42:30.541654+00:00   
3       1004 2024-03-06 07:00:00 2023-02-02 14:42:30.541654+00:00   
4       1005 2024-03-06 08:00:00 2023-02-02 14:42:30.541654+00:00   

   conv_percentage  acc_percentage  
0        10.988791       69.367623  
1        77.247840       87.864637  
2        78.075516        6.504741  
3        44.163507       61.944288  
4        53.096521       52.055502  


In [30]:
query.stop()

In [31]:
import shutil

try:
    shutil.rmtree("/tmp/checkpoint")
except OSError as e:
    print("Error: ", e)